In [1]:
import os, sys, time, resource, re, gc, shutil
import operator
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
from django.db.models import Count
import matplotlib
matplotlib.use('pgf')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
from adjustText import adjust_text

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from django.db.models import F, Sum, Count
from matplotlib import gridspec

from utils.utils import *


import statsmodels.formula.api as sm, statsmodels.stats.api as sms

pgf_with_latex = {
    "text.usetex": True,            # use LaTeX to write all text
    "pgf.rcfonts": False,           # Ignore Matplotlibrc
    "text.latex.unicode": True,
    "pgf.preamble": [
        #r"\usepackage[utf8x]{inputenc}",
        r"\usepackage{xcolor}"
    ],
    "pgf.texsystem" : "xelatex",
    "figure.figsize": [7.2,5]
}
matplotlib.rcParams.update(pgf_with_latex)


from run_id import run_id

run_id = 1056

run_id = 1103

runstat = RunStats.objects.get(pk=run_id)
topics = DynamicTopic.objects.filter(run_id=run_id)

/home/galm/software/django/tmvenv_36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
tsums = topics.aggregate(
    ip_score=Sum('ipcc_score'),
    score=Sum('ipcc_time_score')
)
tsums

topics.update(
    ipcc_share=F('ipcc_score')/tsums['ip_score'],
    share=F('ipcc_time_score')/tsums['score']
)

100

In [3]:
def calculate_deviations(df):
    df['deviation'] = df['ipcc_share'] - df['share']
    df['abs_md'] = abs(df['deviation'])

    md = df['deviation'].max()
    rae = df['abs_md'].mean()
    lh = df['abs_md'].sum() / 2

    df['representation'] = df['ipcc_share'] / df['share']  
    
    df_disp = {'MD':md,'Rae':rae,'L-H':lh}
    return [df,df_disp]


In [4]:



df = pd.DataFrame.from_dict(
    list(topics.values(
        'title',
        'score',
        'ipcc_coverage',
        'share',
        'ipcc_score',
        'ipcc_share',
        'ipcc_time_score',
        'primary_wg'
    ))
)

df, df_disp  = calculate_deviations(df)


df.sort_values('representation').head()

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation
62,0.040841,9.635054,0.001639,235.914874,1,619.386560,0.005924,"{adsorption, capacity, adsorbent}",-0.004285,0.004285,0.276631
57,0.048184,21.109400,0.003590,438.101189,2,1016.456021,0.011001,"{reaction, process, sorbent}",-0.007410,0.007410,0.326365
67,0.049648,9.333797,0.001587,187.998375,3,388.920168,0.004721,"{soc, stock, soil}",-0.003133,0.003133,0.336284
44,0.050752,24.177637,0.004112,476.385707,2,781.243782,0.011962,"{sediment, erosion, deposit}",-0.007850,0.007850,0.343761
91,0.051408,5.756509,0.000979,111.976156,3,358.666053,0.002812,"{membrane, separation, selectivity}",-0.001833,0.001833,0.348205


In [5]:

from matplotlib import ticker


plt.rcParams["figure.figsize"] = [7.2,5]

def plot_representation(df, ax, nts=5, xspace=3,yspace = 0.15, fmin=None, fmax=None):
    
    md = df['deviation'].max()
    rae = df['abs_md'].mean()
    lh = df['abs_md'].sum() / 2

    pdf = df.sort_values('representation',ascending=False).reset_index()#.set_index('title')

    pdf = pdf[pdf['share'] > 0.001]

    pdf['lrep'] = np.log(pdf['representation'])
    #pdf['lrep'] = pdf['representation']
    pdf.set_index('title')['lrep'].plot(kind="bar",ax=ax,color="grey")

    i = 0

    rmax = pdf['lrep'].max()
    if fmax:
        rmax = fmax
    rmin = pdf['lrep'].min()
    if fmin:
        rmin = -fmin
    
    for index, row in pdf.head(nts).iterrows():
        i+=1
        s = round(row['ipcc_share']*100,1)
        v = round(row['share']*100,1)
        ax.annotate(
            s="{} [{},{}]".format(row['title'],s,v),
            xy=(-0.5+i,row['lrep']),
            xytext=((xspace-3)*-1+i*xspace,rmax-rmax*yspace*(i-1)),
            arrowprops=dict(
                facecolor='black', 
                #shrink=0.05,
                width=0.1,
                headwidth=0.2
            ),
            ha="left",
            va="bottom"
        )

    i = 0
    for index, row in pdf.sort_values('representation').head(nts).iterrows():
        i+=1
        s = round(row['ipcc_share']*100,1)
        v = round(row['share']*100,1)
        ax.annotate(
            s="{} [{},{}]".format(row['title'],s,v),
            xy=(len(pdf)-0.5-i,row['lrep']),
            xytext=(len(pdf)+xspace-3-i*xspace,rmin-rmin*yspace*(i-1)),
            arrowprops=dict(
                facecolor='black', 
                #shrink=0.05,
                width=0.1,
                headwidth=0.2
            ),
            ha="right",
            va="top"
        )
        
    #ax.set_yscale('log')
    
    
    if rmin > -1:
        rmin = -1
    if np.exp(rmax) < 2:
        rmax = np.log(2.1)
    
    lmin = int(1/np.exp(rmin)//1)*-1
    lmax = int(np.exp(rmax)//1)
    


    ytick_labels = [i for i in range(lmin,lmax+1) if i not in [-1,1]]
    yticks = []
    for x in ytick_labels:
        if x < 0:
            y = np.log(-1/x)
        elif x==0:
            y = 0
        else:
            y = np.log(x)
        yticks.append(y)
    
    ax.set_yticks(yticks)
    ax.set_yticklabels(ytick_labels)
    
    #ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%.1f"))
        
    rmax = pdf['lrep'].max()
    rmin = pdf['lrep'].min()
    
    ax.text(
        len(pdf)*0.066,-0.2,
        "MD: {:.3f}\nRae: {:.3f}\nL-H: {:.3f}".format(md,rae,lh),
        va="top",
        ha="left",
        bbox={'facecolor':'red', 'alpha':0.3,'pad':10}
   )

    #ax.text(1,-1*0.2,"MD: {:.3f}".format(md))   
    #ax.text(1,-1*0.4,"Rae: {:.3f}".format(rae))   
    #ax.text(1,-1*0.6,"L-H: {:.3f}".format(lh))   
    
    ax.get_xaxis().set_visible(False)#.set_ticks([])
    
    ax.set_ylim((rmin+rmin*0.1,rmax+rmax*0.15))

fig, ax = plt.subplots()
plot_representation(df,ax,nts=10,xspace=6,yspace=0.07)
plt.tight_layout()
plt.savefig('../plots/ipcc_representation/ipcc_rep_{}_all.pdf'.format(run_id),bbox_inches='tight')
plt.show()

In [6]:
rmin = -1.5
rmax = 1
lmin = int(1/np.exp(rmin)//1)*-1
lmax = int(np.exp(rmax)//1)

ytick_labels = [i for i in range(lmin,lmax) if i not in [-1,1]]
print(ytick_labels)
yticks = []
for x in ytick_labels:
    if x < 0:
        x = np.log(-1/x)
    elif x==0:
        x = 0
    else:
        x = np.log(x)
    yticks.append(x)
#yticks = [np.log(x) if x!=0 else 0 for x in ytick_labels ]
print(yticks)

[-4, -3, -2, 0]
[-1.3862943611198906, -1.0986122886681098, -0.6931471805599453, 0]


plt.rcParams["figure.figsize"] = [16,12]

stat = RunStats.objects.get(pk=run_id)

fig = plt.figure()


disp = pd.DataFrame(columns=['AR','MD','Rae','L-H'])
#fig, axs = plt.subplots(2,3,sharey=True)

for i, tp in enumerate(stat.periods.filter(n__lt=6)):
    if i == 0:
        ax = fig.add_subplot(2,3,i+1)
        ax1 = ax
    else:
        ax = fig.add_subplot(2,3,i+1,sharey=ax1)
    print(tp)
    tdts = TimeDTopic.objects.filter(dtopic__run_id=run_id,period=tp)
    
    df = pd.DataFrame.from_dict(
        list(tdts.values(
            'dtopic__title',
            'score',
            'share',
            'ipcc_score',
            'ipcc_share',
            #'ipcc_time_score'
        ))
    )
    
    df = df.rename(columns={'dtopic__title':'title'})
    
    df, df_disp  = calculate_deviations(df)
    df_disp['AR'] = tp.title   
    disp = disp.append(pd.DataFrame.from_dict([df_disp]))

    plot_representation(df, ax, 2.3,1)
    
    ax.set_title(tp.title)
    
    if tp.title=="AR5":
        ax = fig.add_subplot(2,3,6)
        
        disp = disp.set_index('AR')
        
        disp.plot(ax=ax)
        
        ax.set_title("Disproportionality")
    
    
    
    #plt.savefig('../plots/ipcc_representation/ipcc_rep_{}_{}.png'.format(run_id,tp.title),bbox_inches='tight')
    #plt.show()

plt.savefig('../plots/ipcc_representation/ipcc_rep_{}_ARS.png'.format(run_id),bbox_inches='tight')
plt.show()

disp.head()

In [7]:
tds = topics.filter(timedtopic__period__n__lt=6).values(
    'title','timedtopic__period__title','timedtopic__period__n','timedtopic__score','score'
).order_by('id','timedtopic__period__n')

tdf = pd.DataFrame.from_dict(list(tds))

#tdf['ys'] = tdf[]

tdf['share'] = tdf['timedtopic__score'] / tdf['score']

tdf['ys'] = tdf['timedtopic__period__n'] * tdf['share']


tdf.head(12)
#tdf.groupby('')

,score,timedtopic__period__n,timedtopic__period__title,timedtopic__score,title,share,ys
0,580.340559,1,AR1,0.018950,"{farmer, household, farm}",0.000033,0.000033
1,580.340559,2,AR2,0.801268,"{farmer, household, farm}",0.001381,0.002761
2,580.340559,3,AR3,0.393549,"{farmer, household, farm}",0.000678,0.002034
3,580.340559,4,AR4,10.388046,"{farmer, household, farm}",0.017900,0.071600
4,580.340559,5,AR5,106.947070,"{farmer, household, farm}",0.184283,0.921416
5,798.064772,1,AR1,0.190481,"{material, waste, concrete}",0.000239,0.000239
6,798.064772,2,AR2,2.564406,"{material, waste, concrete}",0.003213,0.006427
7,798.064772,3,AR3,13.822033,"{material, waste, concrete}",0.017319,0.051958
8,798.064772,4,AR4,34.809025,"{material, waste, concrete}",0.043617,0.174467
9,798.064772,5,AR5,217.673062,"{material, waste, concrete}",0.272751,1.363756


In [8]:
means = tdf.groupby('title')['ys'].mean()

means = pd.DataFrame({'ys' : tdf.groupby('title')['ys'].mean()}).reset_index()

mdf = df.merge(means)

mdf.head()

means.sort_values('ys',ascending=False).head()



,title,ys
56,"{ozone, stratospheric, tropospheric}",0.645853
66,"{record, glacial, last}",0.574312
45,"{increase, decrease, concentration}",0.565356
24,"{delta, value, isotope}",0.549008
27,"{ecosystem, net, respiration}",0.545490


In [9]:
for name, group in tdf.groupby('title'):
    l = []
    for index, y in group.iterrows():
        print(y.share)
        l = l + [y.timedtopic__period__n] * round(y.share*100)
        
    print(l)
    
    print(np.mean(l))
    
    mdf.loc[mdf['title']==name]['av_year'] = np.mean(l)
    
    break

6.0612956712259736e-05
0.0005912589474106001
0.02018103449063873
0.03364653175813618
0.40570348032881665
[3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
4.8478260869565215


/home/galm/software/django/tmvenv_36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [10]:
def year_av(x):
    group = tdf[tdf['title']==x['title']]
    l = []
    for index, y in group.iterrows():
        l = l + [y.timedtopic__period__n] * round(y.share*100)
    return np.mean(l)

mdf['year_av'] = df.apply(year_av,axis=1)

print(mdf[pd.isna(mdf['year_av'])].shape)

mdf.head()

(0, 13)


,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av
0,0.165563,60.097690,0.010221,362.990459,1,927.170484,0.009115,"{concentration, nutrient, nitrogen}",0.001107,0.001107,1.121410,0.307686,4.162162
1,0.143890,45.553455,0.007748,316.585302,3,757.666212,0.007949,"{price, electricity, market}",-0.000202,0.000202,0.974614,0.373142,4.666667
2,0.237096,162.597845,0.027654,685.790033,1,1085.298548,0.017220,"{sst, pacific, enso}",0.010434,0.010434,1.605926,0.518099,4.423729
3,0.066672,21.648124,0.003682,324.693770,1,594.636481,0.008153,"{gas, hydrate, pressure}",-0.004471,0.004471,0.451594,0.476934,4.444444
4,0.154533,52.515423,0.008932,339.833410,2,662.747728,0.008533,"{heat, wave, stress}",0.000399,0.000399,1.046701,0.443676,4.562500


In [11]:
x = 'year_av'
y = 'representation'

fig, ax = plt.subplots()

cmap = {1: "#66c2a5", 2: "#fc8d62" , 3: "#8da0cb"}
colors = [cmap[i] for i in mdf['primary_wg']]


mdf.plot.scatter(
    x,y,s=mdf['score']*0.03,
    c = colors,ax=ax
)

z = np.polyfit(x=mdf[x], y=mdf[y], deg=1)
p = np.poly1d(z)
mdf['trendline'] = p(mdf.loc[:, x])

plt.plot(mdf[x],mdf['trendline'])

qs = 5

mdf['x_q'] = pd.qcut(mdf[x], qs, labels=False)
mdf['y_q'] = pd.qcut(mdf[y], qs, labels=False)
texts = []


#for i, x in mdf[mdf['year_av']> 5.42].iterrows():
for i, row in mdf[(mdf['x_q'].isin([0,qs-1])) & (mdf['y_q'].isin([0,qs-1]))].iterrows():
    texts.append(plt.text(row[x],row[y],row['title'],ha='center', va='center'))
    
adjust_text(texts, 
            arrowprops=dict(arrowstyle="-", color='black', lw=0.5))

plt.axhline(1,c="grey")
plt.axvline(mdf[x].median(),c="grey")



# Put labels on the quadrants
# x
d1 = mdf[x].median()-plt.xlim()[0]
d2 = mdf[x].median()-plt.xlim()[1]

xlabpoints = [mdf[x].median()-d1/2,mdf[x].median()-d2/2]
xrange = mdf[x].max()-mdf[x].min()
xlabpoints = [mdf[x].min()+xrange*0.15,mdf[x].min()+xrange*0.85]
labels = [
    ['Under-represented,\nolder topics','Under-represented,\nnewer topics'],
    ['Over-represented,\nolder topics','Over-represented,\nnewer topics']
]

pylims = plt.ylim()

tpad = (pylims[1]-pylims[0])*0.13


ops = [operator.lt,operator.gt]


for i in [0,1]:
    for j in [0,1]:
        if j==0:
            tpadx=tpad*-1
        else:
            tpadx=tpad*0.75
            
        q = mdf[(ops[i](mdf[x],mdf[x].median())) & (ops[j](mdf[y],1))]
        qshare = q['score'].sum()/mdf['score'].sum()
        plt.text(
            xlabpoints[i],
            pylims[j]+tpadx,
            labels[j][i] + " {:.0%}".format(qshare),
            va="center",ha="center",
            bbox={'facecolor':'red', 'alpha':qshare*2-0.25, 'pad':10},
            fontsize=7
        )

        

#ax.get_yaxis().set_visible(False)
#ax.tick_params(axis=u'both', which=u'both',length=0)
#ax2.tick_params(axis=u'both', which=u'both',length=0)
plt.xlabel('Assessment period\noccurence')
plt.ylabel('Representation')

plt.tight_layout(h_pad=18)

fig.patch.set_facecolor('#f0f0f0')    
#plt.tight_layout()

plt.savefig(
    '../plots/ipcc_representation/ipcc_rep_new{}_all.pdf'.format(run_id),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

In [12]:
plt.close()

In [13]:
mdf.sort_values('year_av').tail(15)

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,trendline,x_q,y_q
52,0.207929,40.094927,0.006819,192.830327,1,399.760197,0.004842,"{permafrost, arctic, thaw}",0.001977,0.001977,1.408367,0.434247,4.800000,0.939057,4,4
98,0.089752,19.825250,0.003372,220.888852,2,461.960378,0.005546,"{habitat, conservation, biodiversity}",-0.002175,0.002175,0.607920,0.452985,4.808511,0.938193,4,1
60,0.066492,21.357351,0.003632,321.203213,3,710.446606,0.008065,"{reservoir, storage, injection}",-0.004433,0.004433,0.450370,0.426866,4.818182,0.937211,4,0
69,0.124417,31.626393,0.005379,254.197601,1,634.120263,0.006383,"{china, region, province}",-0.001004,0.001004,0.842714,0.377297,4.846154,0.934372,4,2
11,0.288620,64.547701,0.010978,223.642425,2,445.225935,0.005616,"{adaptation, local, mitigation}",0.005362,0.005362,1.954918,0.444978,4.847826,0.934202,4,4
16,0.263921,40.087359,0.006818,151.891796,2,486.579314,0.003814,"{vulnerability, index, vulnerable}",0.003004,0.003004,1.787619,0.281059,4.862069,0.932756,4,4
51,0.149479,50.309418,0.008556,336.565863,2,707.745258,0.008451,"{disease, health, vector}",0.000105,0.000105,1.012468,0.444708,4.866667,0.932289,4,2
58,0.106532,23.658799,0.004024,222.082055,2,474.576178,0.005576,"{fish, fishery, marine}",-0.001553,0.001553,0.721574,0.444514,4.869565,0.931995,4,1
62,0.040841,9.635054,0.001639,235.914874,1,619.386560,0.005924,"{adsorption, capacity, adsorbent}",-0.004285,0.004285,0.276631,0.370600,4.894737,0.929439,4,0
90,0.158361,19.538718,0.003323,123.381002,2,580.340559,0.003098,"{farmer, household, farm}",0.000225,0.000225,1.072629,0.199569,4.900000,0.928905,4,3


In [14]:
mdf[mdf[y].isna()]

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,trendline,x_q,y_q
